In [1]:
import os
import numpy as np
import pandas as pd

# Extract Data

In [2]:
df = pd.read_csv('Occupational_Employment_and_Wage_Statistics.csv')

In [3]:
df.head()

,Area Type,Area,Area Name,Standard Occupational Code,Occupational Title,Employment,Mean Wage,Median Wage,Entry Wage,Experienced Wage
0,1,36,New York State,00-0000,"Total, All Occupations","9,108,930","77,445","54,276","34,849","98,743"
1,1,36,New York State,11-0000,Management Occupations,"567,480","173,530","142,285","77,836","221,378"
2,1,36,New York State,11-1011,Chief Executives,"4,910","330,808","233,777","126,304","433,060"
3,1,36,New York State,11-1021,General and Operations Managers,"212,710","160,384","126,364","64,477","208,338"
4,1,36,New York State,11-1031,Legislators,"4,530","110,711","99,645","50,788","140,674"


In [4]:
df.shape

(7010, 10)

# Data Clean

In [5]:
missing_values = df.isna().sum()
missing_values

Area Type                                                                                                                                                                                                                                                            0
 Area                                                                                                                                                                                                                                                                0
 Area Name                                                                                                                                                                                                                                                           0
 Standard Occupational Code                                                                                                                                                                                        

In [6]:
cleaned_data = df.dropna()

In [7]:
cleaned_data.shape

(5369, 10)

In [8]:
cleaned_data.dtypes

Area Type                                                                                                                                                                                                                                                          int64
 Area                                                                                                                                                                                                                                                              int64
 Area Name                                                                                                                                                                                                                                                        object
 Standard Occupational Code                                                                                                                                                                                  

In [9]:
new_cols = [i.strip() for i in cleaned_data.columns.tolist()]
cleaned_data.columns = new_cols

In [10]:
new_cols

['Area Type',
 'Area',
 'Area Name',
 'Standard Occupational Code',
 'Occupational Title',
 'Employment',
 'Mean Wage',
 'Median Wage',
 'Entry Wage',
 'Experienced Wage']

In [11]:
# convert column data into whole number
columns_to_convert = ['Employment', 'Mean Wage', 'Median Wage', 'Entry Wage', 'Experienced Wage']

In [12]:
for col in columns_to_convert:
    print(col)
    cleaned_data[col] =cleaned_data[col].str.replace(',', '').astype(int)

Employment
Mean Wage
Median Wage
Entry Wage
Experienced Wage


/var/folders/ng/8w50yqg14d76t2g856pwdxf40000gn/T/ipykernel_7446/3229129351.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data[col] =cleaned_data[col].str.replace(',', '').astype(int)


In [13]:
cleaned_data.dtypes  # confirm data is ready to load

Area Type                      int64
Area                           int64
Area Name                     object
Standard Occupational Code    object
Occupational Title            object
Employment                     int64
Mean Wage                      int64
Median Wage                    int64
Entry Wage                     int64
Experienced Wage               int64
dtype: object

# Data Load

In [14]:
import pymongo
from pymongo import MongoClient

In [15]:
# connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['Occupational_Employment_and_Wage_Statistics']
collection = db['data_collection']

In [16]:
import json
# convert cleaned data frame to JSON format
#json_data = cleaned_data.to_json(orient='records', force_ascii=False)
#json_list = json.loads(json_data)

# load it into database
#collection.insert_many(json_list)

In [17]:
# test data
collection.find_one({})

{'_id': ObjectId('65778c6a5be31a43448ff610'),
 'Area Type': 1,
 'Area': 36,
 'Area Name': 'New York State',
 'Standard Occupational Code': '00-0000',
 'Occupational Title': 'Total, All Occupations',
 'Employment': 9108930,
 'Mean Wage': 77445,
 'Median Wage': 54276,
 'Entry Wage': 34849,
 'Experienced Wage': 98743}

# Start API Server

In [1]:
from flask import Flask, request, jsonify
import pymongo
from pymongo import MongoClient

#start API service
app = Flask(__name__) 

# connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['Occupational_Employment_and_Wage_Statistics']
collection = db['data_collection']


@app.route('/api/data', methods=['GET'])
def get_data():
    # get data from API
    items = list(collection.find({},{'_id':0}))
    # return in JSON format
    return jsonify(items)    


@app.route('/api/data_area_employment', methods=['GET'])
def get_data_area_employment():
    # define pipeline
    pipeline = [
        {
            '$group': {
                '_id': '$Area',
                'total_employment': {
                    '$sum': '$Employment'
                }
            }
        },
        {
            '$sort': {
                'total_employment': -1
            }
        }
    ]
    # aggregate pipeline
    result = collection.aggregate(pipeline)
    return jsonify(list(result))   


@app.route('/api/data_by_occupation', methods=['GET'])
def get_data_by_occupation():
    occupation_title = request.args.get('occupation_title')
    order_by = request.args.get('order_by', 'Median Wage')
    
    sort_column = None
    if order_by == 'Experienced Wage':
        sort_column = 'Experienced Wage'
    elif order_by == 'Median Wage':
        sort_column = 'Median Wage'
    elif order_by == 'Entry Wage':
        sort_column = 'Entry Wage'
    
    query = {'Occupational Title': occupation_title}
    
    if sort_column:
        items = list(collection.find(query, {'_id': 0}).sort(sort_column, pymongo.DESCENDING))
    else:
        # Handle cases where an invalid sorting column is provided
        return jsonify({'error': 'Invalid sorting column provided'}), 400
    
    
    return jsonify(items)



@app.route('/api/median_wage_statistics', methods=['GET'])
def get_median_wage_statistics():
    area_or_occupation = request.args.get('area_or_occupation')
    
    if area_or_occupation == 'area':
        # Provide median wage statistics by area
        pipeline = [
            {
                '$group': {
                    '_id': '$Area',
                    'median_wage': {
                        '$avg': '$Median Wage'
                    }
                }
            },
            {
                '$sort': {
                    'median_wage': -1
                }
            }
        ]
    else:
        # Provide median wage statistics by occupation
        pipeline = [
            {
                '$group': {
                    '_id': '$Occupational Title',
                    'median_wage': {
                        '$avg': '$Median Wage'
                    }
                }
            },
            {
                '$sort': {
                    'median_wage': -1
                }
            }
        ]
    
    result = collection.aggregate(pipeline)
    return jsonify(list(result))


# run Flask API
if __name__ == '__main__':
    app.run(debug=True)



In [2]:
app.run(debug=True)  # start server

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
Traceback (most recent call last):
  File "/Users/james/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 15, in <module>
    from ipykernel import kernelapp as app
  File "/Users/james/anaconda3/lib/python3.11/site-packages/ipykernel/__init__.py", line 5, in <module>
    from .connect import *  # noqa
    ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/james/anaconda3/lib/python3.11/site-packages/ipykernel/connect.py", line 11, in <module>
    import jupyter_client
  File "/Users/james/anaconda3/lib/python3.11/site-packages/jupyter_client/__init__.py", line 8, in <module>
    from .asynchronous import AsyncKernelClient  # noqa
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/james/anaconda3/lib/python3.11/site-packages/jupyter_client/asynchronous/__init__.py", line 1, in <module>
    from .client import AsyncKernelClient  # noqa
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

SystemExit: 1

/Users/james/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# API

In [18]:
import requests

##  data

In [8]:
# http://127.0.0.1:5000/api/data   # python app.py runserver in terminal to open server

In [19]:
res = requests.get('http://127.0.0.1:5000/api/data')  # extract all the data

In [20]:
df = pd.DataFrame(res.json())  # convert data into JSON format

In [21]:
df.head()

,Area,Area Name,Area Type,Employment,Entry Wage,Experienced Wage,Mean Wage,Median Wage,Occupational Title,Standard Occupational Code
0,36,New York State,1,9108930,34849,98743,77445,54276,"Total, All Occupations",00-0000
1,36,New York State,1,567480,77836,221378,173530,142285,Management Occupations,11-0000
2,36,New York State,1,4910,126304,433060,330808,233777,Chief Executives,11-1011
3,36,New York State,1,212710,64477,208338,160384,126364,General and Operations Managers,11-1021
4,36,New York State,1,4530,50788,140674,110711,99645,Legislators,11-1031


## pipeline

In [22]:
res = requests.get('http://127.0.0.1:5000/api/data_area_employment')

In [23]:
df_new = pd.DataFrame(res.json())
df_new

,_id,total_employment
0,36,27105700
1,7,12550250
2,5,3718760
3,4,2638190
4,10,1772680
5,3,1563000
6,1,1464170
7,2,960470
8,9,708830
9,6,521040


In [25]:
res = requests.get('http://127.0.0.1:5000/api/median_wage_statistics')
df_new = pd.DataFrame(res.json())
df_new

,_id,median_wage
0,Emergency Medicine Physicians,241959.000000
1,Anesthesiologists,226573.000000
2,Obstetricians and Gynecologists,224979.000000
3,"Ophthalmologists, Except Pediatric",219746.500000
4,Nurse Anesthetists,213904.000000
...,...,...
700,Taxi Drivers,31548.666667
701,"Lifeguards, Ski Patrol, and Other Recreational...",31328.454545
702,Manicurists and Pedicurists,31295.000000
703,"Door-to-Door Sales Workers, News and Street Ve...",31273.500000


In [26]:
res = requests.get('http://127.0.0.1:5000/api/median_wage_statistics?area_or_occupation=area')
df_new = pd.DataFrame(res.json())
df_new

,_id,median_wage
0,7,81337.891544
1,5,76824.005693
2,4,74585.808824
3,36,73253.590705
4,1,64442.816327
5,3,63282.789981
6,2,63008.571749
7,10,62815.691149
8,9,61195.087940
9,8,59500.496368


In [27]:
res = requests.get('http://127.0.0.1:5000/api/data_by_occupation?occupation_title=Software Developers&order_by=Entry Wage')
df_new = pd.DataFrame(res.json())
df_new

,Area,Area Name,Area Type,Employment,Entry Wage,Experienced Wage,Mean Wage,Median Wage,Occupational Title,Standard Occupational Code
0,7,New York City,10,60630,101527,186721,158323,146062,Software Developers,15-1252
1,36,New York State,1,92040,88508,175572,146551,140280,Software Developers,15-1252
2,4,Hudson Valley,10,4580,85558,163600,137585,133635,Software Developers,15-1252
3,5,Long Island,10,6690,83101,163582,136755,135488,Software Developers,15-1252
4,2,Central New York,10,1780,77282,141218,119906,112984,Software Developers,15-1252
5,3,Finger Lakes,10,4230,73499,136523,115515,111796,Software Developers,15-1252
6,1,Capital Region,10,3060,72356,136021,114799,109445,Software Developers,15-1252
7,10,Western New York,10,2920,70876,134231,113114,109747,Software Developers,15-1252
8,9,Southern Tier,10,1630,68759,135822,113468,111631,Software Developers,15-1252
9,6,Mohawk Valley,10,630,67354,130618,109530,105462,Software Developers,15-1252
